In [1]:
import sgp4
from sgp4.api import Satrec, jday
import numpy as np
import datetime as dt
import time
import pyproj
import dask
from dask import bag as db
from dask.distributed import Client,LocalCluster

In [ ]:
!pip install sgp4 --quiet
!pip install pyproj --quiet

1. Text file has Tle lines we have to obtain the co-ordinates for one day with time frame of one minute.(get_satellite_positions)
2. Convert the co-ordinates to longitude, latitude and laltitude.(ecef2lla,convert_to_lla_single)
3. Compare within a given range of latitude and longitude.(filter_positions)
4. Code Optimization with Distributed Computing.(Using Dask Library)


In [2]:
%%time

def create_satellite_objects(tle_lines):
    #sgp4 Library - Data Source Library.
    satellites = []
    for line in tle_lines:
        satellite = Satrec.twoline2rv(line[1], line[2])
        satellites.append([satellite,line[0]])
    return satellites

def get_satellite_positions(satellites, start_time, end_time, interval):
    sat_positions = []
    error_satpositions = []
    delta = dt.timedelta(minutes=interval)
    current_time = start_time

    while current_time <= end_time:
        jd, fr = jday(current_time.year, current_time.month, current_time.day,
                      current_time.hour, current_time.minute, current_time.second)
        for satellite in satellites:
            e, r, v = satellite[0].sgp4(jd, fr)
            if e == 0:
                sat_positions.append([satellite[1],current_time, r, v])
            elif not e==0 :
                error_satpositions.append([satellite[1],current_time, r, v])
                
        current_time += delta
    return sat_positions,error_satpositions

# Load TLEs
with open(r"C:\Users\yaswa\Downloads\Assignment_Code Optimization\Assignment_Code Optimization\Assignment_Code Optimization\30000sats.txt", 'r') as file:
    lines = file.readlines()
tle_lines = [(lines[i], lines[i + 1],lines[i+2]) for i in range(0, len(lines), 3)]

# Create satellite objects
satellites = create_satellite_objects(tle_lines)

# Get positions for one day
start_time = dt.datetime(2024, 1, 1, 0, 0, 0)
end_time = dt.datetime(2024, 1, 1, 23, 59, 59)
positions = get_satellite_positions(satellites, start_time, end_time, 1)[0]


CPU times: total: 10.5 s
Wall time: 23.4 s


In [ ]:
#get_satellite_positions(satellites, start_time, end_time, 1)[1]

In [3]:
transformer = pyproj.Transformer.from_proj(
    pyproj.Proj(proj="geocent", ellps="WGS84", datum="WGS84"),
    pyproj.Proj(proj="latlong", ellps="WGS84", datum="WGS84"),
    always_xy=True
)

In [ ]:
%time
def ecef2lla(pos_x, pos_y, pos_z,transformer):
    #Pyproj Library - Computation Library.
    lon, lat, alt = transformer.transform(pos_x, pos_y, pos_z, radians=False)
    return lon, lat, alt

def convert_to_lla_single(position,transformer):

    name, timestamp, r, v = position
    lon, lat, alt = ecef2lla(r[0], r[1], r[2],transformer)

    return name, timestamp, lon, lat, alt

def filter_positions(lla_positions, lat_min, lat_max, lon_min, lon_max):
    filtered_positions = []
    for time, lon, lat, alt in lla_positions:
        if lat_min <= lat <= lat_max and lon_min <= lon <= lon_max:
            filtered_positions.append((time, lon, lat, alt))
    return filtered_positions



In [4]:
a = len(positions)//3
positions = positions[:a]

In [5]:
transformer

<Conversion Transformer: pipeline>
Description: Conversion from unknown (geocentric) to unknown (geog2D)
Area of Use:
- undefined

In [ ]:
27360*0.018

In [ ]:
import pyproj
from pyproj import Transformer

import time
start = time.time()
def ecef2lla(pos_x, pos_y, pos_z,transformer):
    #start = time.time()
    
    lon, lat, alt = transformer.transform(pos_x, pos_y, pos_z, radians=False)
    #end = time.time()
    #print(end -start)
    return lon, lat, alt




def convert_to_lla(positions,transformer):
    
    lla_positions = []
    for name,time, r, v in positions:
        lon, lat, alt = ecef2lla(r[0], r[1], r[2],transformer)
        lla_positions.append([name,time, lon, lat, alt])

    return lla_positions

lla_positions = convert_to_lla(positions,transformer)
def filter_positions(lla_positions, lat_min, lat_max, lon_min, lon_max):
    filtered_positions = []
    for name,time, lon, lat, alt in lla_positions:
        if lat_min <= lat <= lat_max and lon_min <= lon <= lon_max:
            filtered_positions.append((time, lon, lat, alt))
    return filtered_positions

lat_min, lat_max = -1000.09096, 1000.74973
lon_min, lon_max = -1470.79778, 1030.58196
filtered_positions = filter_positions(lla_positions, lat_min, lat_max, lon_min, lon_max)

end =time.time()
print(end-start)

In [ ]:
len(filtered_positions)

In [ ]:
lla_positions

In [ ]:
filtered_positions

# Distributed Computing


In [ ]:
!pip install dask-labextension==6.2.0 --quiet #For Good Dashboards using Jupyter Lab

In [6]:
cluster = LocalCluster()
cluster

In [ ]:
#help(cluster)

In [7]:
client = Client(cluster)

In [ ]:
start = time.time()
def convert_and_filter(positions,transformer, lat_min, lat_max, lon_min, lon_max, use_multithreading=True, n_workers=8):
   
    # Create a Dask bag from the positions
    tasks = db.from_sequence(positions)

    # Map the conversion function to each position
    lla_positions = tasks.map(lambda pos: convert_to_lla_single(pos, transformer))

    # Filter the results
    filtered_positions = lla_positions.filter(lambda pos: lat_min <= pos[3] <= lat_max and lon_min <= pos[2] <= lon_max)

    # Compute the filtered positions
    results = filtered_positions.compute()

    # Close the Dask client
    

    return results

lat_min, lat_max = -1001,1000
lon_min, lon_max = -1000,1000  
lla_positions_filtered_mt = convert_and_filter(positions, lat_min, lat_max, lon_min, lon_max, n_workers=8)
end = time.time()
print(end-start)

In [ ]:
client.close()
cluster.close()